# Steps

merge
- merge CHR with fips code using 'cbsacode' column
- merge SMART with CHR
- select columns
- aggregate the data

In [24]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [25]:
intermediate_dir = "../00_data/02_intermediate/"

In [26]:
chr = pd.read_csv(f"{intermediate_dir}CHR.csv")[1:]
smart_cbsa = pd.read_csv(f"{intermediate_dir}smart_with_cbsa_all_12_years.csv")#, usecols=lambda column: column != 'Unnamed: 0') ## change
smart_cbsa['cbsacode'] = smart_cbsa['cbsacode'].astype(int)
cbsa = pd.read_csv('https://data.nber.org/cbsa-csa-fips-county-crosswalk/cbsa2fipsxw.csv')

In [27]:
smart_cbsa.shape

(876, 8)

In [28]:
chr.head(3)

,county,countycode,fipscode,state,statecode,year,v051_rawvalue,v052_rawvalue,statecode.1,countycode.1,fipscode.1,state.1,county.1,year.1,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
1,Autauga County,1,1001.0,AL,1,2011.0,50756.0,14110.168,1,1,1001.0,AL,Autauga County,2011.0,10040.957968,11543.437080,5167.062312,4177.624848,NaN,7660.0,2143.0,4119.0,1.0,51622.0,19529.0,307.8
2,Baldwin County,3,1003.0,AL,1,2011.0,179878.0,41551.818,1,3,1003.0,AL,Baldwin County,2011.0,30293.433858,36241.459684,26835.279308,13555.965836,NaN,26292.0,7094.0,11557.0,29.0,51957.0,63562.0,226.4
3,Barbour County,5,1005.0,AL,1,2011.0,29737.0,6631.351,1,5,1005.0,AL,Barbour County,2011.0,5291.193621,8687.724024,1963.980165,3142.368264,NaN,2869.0,1271.0,3569.0,2.0,30896.0,8347.0,474.3


In [29]:
smart_cbsa.head(3)

,cbsacode,Year,cbsatitle,Data_value_unit,Data_value,Confidence_limit_Low,Confidence_limit_High,Sample_Size
0,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880
1,10740,2012,"Albuquerque, NM",%,21.22,19.48,22.96,2876
2,10740,2013,"Albuquerque, NM",%,20.25,18.05,22.45,1748


In [31]:
len(smart_cbsa.cbsacode.unique())

73

In [32]:
smart_cbsa.groupby("Year").size()

Year
2011    73
2012    73
2013    73
2014    73
2015    73
2016    73
2017    73
2018    73
2019    73
2020    73
2021    73
2022    73
dtype: int64

In [33]:
smart_cbsa.columns

Index(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit', 'Data_value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Sample_Size'],
      dtype='object')

## Merge

In [34]:
print(smart_cbsa.shape, chr.shape)

(876, 8) (38321, 26)


In [35]:
chr['fipscode'].nunique()

3200

In [36]:

cbsa['metropolitandivisioncode'] = cbsa['metropolitandivisioncode'].fillna(cbsa['cbsacode']) # brutal force, alternatively. could dropna()
cbsa['fipscode'] = cbsa.fipsstatecode.astype(str) + cbsa.fipscountycode.astype(str).str.zfill(3)

cbsa = cbsa[["fipscode", "cbsacode"]] #, "metropolitandivisioncode","csacode", "cbsatitle", "metropolitanmicropolitanstatis", "fipscode"]]
cbsa.head(3)

,fipscode,cbsacode
0,1001,33860
1,1003,19300
2,1005,21640


In [37]:
chr

,county,countycode,fipscode,state,statecode,year,v051_rawvalue,v052_rawvalue,statecode.1,countycode.1,fipscode.1,state.1,county.1,year.1,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
1,Autauga County,1,1001.0,AL,1,2011.0,50756.0,14110.168000,1,1,1001.0,AL,Autauga County,2011.0,10040.957968,11543.437080,5167.062312,4177.624848,NaN,7660.0,2143.0,4119.0,1.0,51622.0,19529.0,307.8
2,Baldwin County,3,1003.0,AL,1,2011.0,179878.0,41551.818000,1,3,1003.0,AL,Baldwin County,2011.0,30293.433858,36241.459684,26835.279308,13555.965836,NaN,26292.0,7094.0,11557.0,29.0,51957.0,63562.0,226.4
3,Barbour County,5,1005.0,AL,1,2011.0,29737.0,6631.351000,1,5,1005.0,AL,Barbour County,2011.0,5291.193621,8687.724024,1963.980165,3142.368264,NaN,2869.0,1271.0,3569.0,2.0,30896.0,8347.0,474.3
4,Bibb County,7,1007.0,AL,1,2011.0,21587.0,5029.771000,1,7,1007.0,AL,Bibb County,2011.0,5463.885570,5347.984967,1870.966877,1837.852419,NaN,2497.0,1030.0,1635.0,0.0,41076.0,7274.0,300.5
5,Blount County,9,1009.0,AL,1,2011.0,58345.0,14119.490000,1,9,1009.0,AL,Blount County,2011.0,9862.288730,14107.937690,2167.049990,5041.708140,NaN,6599.0,2348.0,3343.0,1.0,46086.0,19290.0,104.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38317,Sweetwater County,37,56037.0,WY,56,2022.0,42673.0,10843.000002,56,37,56037.0,WY,Sweetwater County,2022.0,5411.100000,10663.049999,5612.432067,2641.890000,NaN,7490.0,1532.0,1774.0,105.0,70583.0,16120.0,349.5
38318,Teton County,39,56039.0,WY,56,2022.0,23497.0,4161.000001,56,39,56039.0,WY,Teton County,2022.0,2184.968000,4292.592000,3745.177664,1276.176000,NaN,6344.0,912.0,773.0,128.0,92488.0,9685.0,332.4
38319,Uinta County,41,56041.0,WY,56,2022.0,20215.0,5608.000000,56,41,56041.0,WY,Uinta County,2022.0,2746.116000,4352.886000,2656.623943,1212.381000,NaN,3015.0,582.0,628.0,74.0,71246.0,7653.0,267.0
38320,Washakie County,43,56043.0,WY,56,2022.0,7760.0,1706.000000,56,43,56043.0,WY,Washakie County,2022.0,1065.504000,1749.606000,1154.502489,538.806000,NaN,1137.0,211.0,292.0,31.0,58532.0,3182.0,192.2


In [38]:
print("chr.shape", chr.shape)
chr = chr.dropna(subset=['fipscode']).copy() ## brutal
print("chr.shape", chr.shape)

print(cbsa.shape, chr.shape)

chr['fipscode'] = chr['fipscode'].astype(int)
cbsa['fipscode'] = cbsa['fipscode'].astype(int)

chr_cbsa = pd.merge(
    cbsa, chr, 
    on='fipscode', 
    how='inner' ##
)

chr_cbsa.shape

chr.shape (38321, 26)
chr.shape (38320, 26)
(1915, 2) (38320, 26)


(22020, 27)

In [39]:
# sanity check
chr['fipscode'] = chr['fipscode'].astype(int)
cbsa['fipscode'] = cbsa['fipscode'].astype(int)

merged_df = pd.merge(
    cbsa, chr,
    left_on='fipscode', 
    right_on='fipscode',
    indicator=True 
)

print("merged_df.shape:", merged_df.shape)
merged_df
left_only_rows = merged_df[merged_df['_merge'] == 'left_only']

print("Left-only rows in 'chr':")
print(len(left_only_rows))

merged_df.shape: (22020, 28)
Left-only rows in 'chr':
0


In [40]:
print(chr_cbsa.shape, smart_cbsa.shape)

(22020, 27) (876, 8)


In [41]:
smart_cbsa.cbsacode

0      10740
1      10740
2      10740
3      10740
4      10740
       ...  
871    49340
872    49340
873    49340
874    49340
875    49340
Name: cbsacode, Length: 876, dtype: int64

In [42]:
print("Columns in smart_cbsa:", smart_cbsa.columns)
print("Columns in chr:", chr_cbsa.columns)

Columns in smart_cbsa: Index(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit', 'Data_value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Sample_Size'],
      dtype='object')
Columns in chr: Index(['fipscode', 'cbsacode', 'county', 'countycode', 'state', 'statecode',
       'year', 'v051_rawvalue', 'v052_rawvalue', 'statecode.1', 'countycode.1',
       'fipscode.1', 'state.1', 'county.1', 'year.1', 'v009_rawvalue',
       'v011_rawvalue', 'v049_rawvalue', 'v060_rawvalue', 'v070_numerator',
       'v069_numerator', 'v023_numerator', 'v082_numerator', 'v062_numerator',
       'v063_rawvalue', 'v067_numerator', 'v045_rawvalue'],
      dtype='object')


In [43]:
for i in (chr_cbsa.columns):
    print(i)

fipscode
cbsacode
county
countycode
state
statecode
year
v051_rawvalue
v052_rawvalue
statecode.1
countycode.1
fipscode.1
state.1
county.1
year.1
v009_rawvalue
v011_rawvalue
v049_rawvalue
v060_rawvalue
v070_numerator
v069_numerator
v023_numerator
v082_numerator
v062_numerator
v063_rawvalue
v067_numerator
v045_rawvalue


In [46]:
chr_cbsa.to_csv('../00_data/02_intermediate/chr_cbsa.csv',index=False)

In [121]:
data = pd.merge(
    smart_cbsa, chr_cbsa,
    left_on=['cbsacode', 'Year'],
    right_on=['cbsacode', 'year'],
    how='left'
)
data.head(3)

,cbsacode,Year,cbsatitle,Data_value_unit,Data_value,Confidence_limit_Low,Confidence_limit_High,Sample_Size,fipscode,county,countycode,state,statecode,year,v051_rawvalue,v052_rawvalue,statecode.1,countycode.1,fipscode.1,state.1,county.1,year.1,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
0,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35001.0,Bernalillo County,1.0,NM,35.0,2011.0,642527.0,156134.061,35.0,1.0,35001.0,NM,Bernalillo County,2011.0,90469.086654,96792.194861,63717.475009,27724.397523,NaN,112763.0,22525.0,55430.0,156.0,46988.0,233811.0,508.4
1,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35043.0,Sandoval County,43.0,NM,35.0,2011.0,125988.0,32756.880,35.0,43.0,35043.0,NM,Sandoval County,2011.0,15849.290400,21722.850960,12772.663440,5873.560560,NaN,19498.0,4607.0,9851.0,15.0,56479.0,41843.0,255.9
2,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35057.0,Torrance County,57.0,NM,35.0,2011.0,16475.0,3921.050,35.0,57.0,35057.0,NM,Torrance County,2011.0,4042.371900,2887.408500,1719.891150,765.790950,NaN,1525.0,626.0,1498.0,0.0,36419.0,5102.0,215.1


In [122]:
data.columns

Index(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit', 'Data_value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Sample_Size',
       'fipscode', 'county', 'countycode', 'state', 'statecode', 'year',
       'v051_rawvalue', 'v052_rawvalue', 'statecode.1', 'countycode.1',
       'fipscode.1', 'state.1', 'county.1', 'year.1', 'v009_rawvalue',
       'v011_rawvalue', 'v049_rawvalue', 'v060_rawvalue', 'v070_numerator',
       'v069_numerator', 'v023_numerator', 'v082_numerator', 'v062_numerator',
       'v063_rawvalue', 'v067_numerator', 'v045_rawvalue'],
      dtype='object')

In [123]:
data = data.loc[:, ~data.columns.str.endswith('.1')]


In [124]:
data.shape

(6060, 28)

In [125]:
data.groupby(['Year']).size()

Year
2011    505
2012    505
2013    505
2014    505
2015    505
2016    505
2017    505
2018    505
2019    505
2020    505
2021    505
2022    505
dtype: int64

In [126]:
data.groupby(['Year', 'cbsacode']).size()

Year  cbsacode
2011  10740        4
      11260        2
      12060       29
      12260        7
      12420        5
                  ..
2022  45820        5
      46140        7
      47260       18
      48620        4
      49340        1
Length: 876, dtype: int64

In [127]:
data.columns

Index(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit', 'Data_value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Sample_Size',
       'fipscode', 'county', 'countycode', 'state', 'statecode', 'year',
       'v051_rawvalue', 'v052_rawvalue', 'v009_rawvalue', 'v011_rawvalue',
       'v049_rawvalue', 'v060_rawvalue', 'v070_numerator', 'v069_numerator',
       'v023_numerator', 'v082_numerator', 'v062_numerator', 'v063_rawvalue',
       'v067_numerator', 'v045_rawvalue'],
      dtype='object')

In [128]:
for i in data.columns:
    print(i)

cbsacode
Year
cbsatitle
Data_value_unit
Data_value
Confidence_limit_Low
Confidence_limit_High
Sample_Size
fipscode
county
countycode
state
statecode
year
v051_rawvalue
v052_rawvalue
v009_rawvalue
v011_rawvalue
v049_rawvalue
v060_rawvalue
v070_numerator
v069_numerator
v023_numerator
v082_numerator
v062_numerator
v063_rawvalue
v067_numerator
v045_rawvalue


In [129]:
data.head(3)

,cbsacode,Year,cbsatitle,Data_value_unit,Data_value,Confidence_limit_Low,Confidence_limit_High,Sample_Size,fipscode,county,countycode,state,statecode,year,v051_rawvalue,v052_rawvalue,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
0,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35001.0,Bernalillo County,1.0,NM,35.0,2011.0,642527.0,156134.061,90469.086654,96792.194861,63717.475009,27724.397523,NaN,112763.0,22525.0,55430.0,156.0,46988.0,233811.0,508.4
1,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35043.0,Sandoval County,43.0,NM,35.0,2011.0,125988.0,32756.880,15849.290400,21722.850960,12772.663440,5873.560560,NaN,19498.0,4607.0,9851.0,15.0,56479.0,41843.0,255.9
2,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35057.0,Torrance County,57.0,NM,35.0,2011.0,16475.0,3921.050,4042.371900,2887.408500,1719.891150,765.790950,NaN,1525.0,626.0,1498.0,0.0,36419.0,5102.0,215.1


In [130]:
data.columns

Index(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit', 'Data_value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Sample_Size',
       'fipscode', 'county', 'countycode', 'state', 'statecode', 'year',
       'v051_rawvalue', 'v052_rawvalue', 'v009_rawvalue', 'v011_rawvalue',
       'v049_rawvalue', 'v060_rawvalue', 'v070_numerator', 'v069_numerator',
       'v023_numerator', 'v082_numerator', 'v062_numerator', 'v063_rawvalue',
       'v067_numerator', 'v045_rawvalue'],
      dtype='object')

In [131]:
columns_to_drop = ['state',
                    'county',
                    'year','countycode']
filtered_df = data.drop(columns=columns_to_drop)

new_column_names = {
    'fipscode': 'smart_fipscode',
}

# Rename columns in filtered_df
filtered_df = filtered_df.rename(columns=new_column_names)


In [132]:
for i in filtered_df.columns:
    print(i)

cbsacode
Year
cbsatitle
Data_value_unit
Data_value
Confidence_limit_Low
Confidence_limit_High
Sample_Size
smart_fipscode
statecode
v051_rawvalue
v052_rawvalue
v009_rawvalue
v011_rawvalue
v049_rawvalue
v060_rawvalue
v070_numerator
v069_numerator
v023_numerator
v082_numerator
v062_numerator
v063_rawvalue
v067_numerator
v045_rawvalue


In [133]:
sum_cols = [col for col in filtered_df.columns if col.startswith('v')]
#sum_cols.remove("v052_rawvalue")  # Remove the rate% of children to pop
sum_cols

['v051_rawvalue',
 'v052_rawvalue',
 'v009_rawvalue',
 'v011_rawvalue',
 'v049_rawvalue',
 'v060_rawvalue',
 'v070_numerator',
 'v069_numerator',
 'v023_numerator',
 'v082_numerator',
 'v062_numerator',
 'v063_rawvalue',
 'v067_numerator',
 'v045_rawvalue']

In [134]:
filtered_df.head(2)

,cbsacode,Year,cbsatitle,Data_value_unit,Data_value,Confidence_limit_Low,Confidence_limit_High,Sample_Size,smart_fipscode,statecode,v051_rawvalue,v052_rawvalue,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
0,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35001.0,35.0,642527.0,156134.061,90469.086654,96792.194861,63717.475009,27724.397523,NaN,112763.0,22525.0,55430.0,156.0,46988.0,233811.0,508.4
1,10740,2011,"Albuquerque, NM",%,21.78,19.92,23.64,2880,35043.0,35.0,125988.0,32756.880,15849.290400,21722.850960,12772.663440,5873.560560,NaN,19498.0,4607.0,9851.0,15.0,56479.0,41843.0,255.9


In [135]:
filtered_df.columns

Index(['cbsacode', 'Year', 'cbsatitle', 'Data_value_unit', 'Data_value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Sample_Size',
       'smart_fipscode', 'statecode', 'v051_rawvalue', 'v052_rawvalue',
       'v009_rawvalue', 'v011_rawvalue', 'v049_rawvalue', 'v060_rawvalue',
       'v070_numerator', 'v069_numerator', 'v023_numerator', 'v082_numerator',
       'v062_numerator', 'v063_rawvalue', 'v067_numerator', 'v045_rawvalue'],
      dtype='object')

In [136]:
def weighted_sum_proportional(group):
    result_value = {}
    for col in sum_cols:
        total_population = group['v051_rawvalue'].fillna(0).sum()
        weighted_sum = group[col].fillna(0).sum()
        sum_population_non_na = group.loc[group[col].notna(), 'v051_rawvalue'].fillna(0).sum()
        result_value[col] = (weighted_sum / sum_population_non_na) * total_population if sum_population_non_na != 0 else 0
    return pd.Series(result_value)


aggregated_df = filtered_df.groupby([
    'Year', "cbsacode", 'Sample_Size', 'Data_value', 'Data_value_unit'
]).apply(weighted_sum_proportional).reset_index()


aggregated_df['Year'] = aggregated_df['Year'].astype(int)
print(aggregated_df.shape)

(876, 19)


/var/folders/xn/k3tchqyx5bn081b3tlx0k4940000gn/T/ipykernel_42338/1444736341.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated_df = filtered_df.groupby([


In [137]:
aggregated_df

,Year,cbsacode,Sample_Size,Data_value,Data_value_unit,v051_rawvalue,v052_rawvalue,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue
0,2011,10740,2880,21.78,%,857903.0,2.117694e+05,123687.787094,1.357007e+05,85547.993919,38302.509293,0.0,143682.0,30309.0,73613.0,174.0,182856.0,303398.0,1239.8
1,2011,11260,482,17.54,%,374553.0,9.842329e+04,58399.185228,7.407142e+04,49694.181724,18560.182330,0.0,70584.0,14187.0,27308.0,147.0,145971.0,132468.0,1110.0
2,2011,12060,15892,13.45,%,5503952.0,1.483403e+06,688901.435410,1.063977e+06,605445.249041,382898.985792,0.0,1066375.0,260002.0,457387.0,1928.0,1608146.0,1942529.0,8768.2
3,2011,12260,1442,15.61,%,547067.0,1.387515e+05,88740.928716,1.282184e+05,56390.564574,46079.670066,0.0,78056.0,24444.0,56538.0,156.0,298566.0,185331.0,3372.0
4,2011,12420,1205,15.76,%,1705075.0,4.299228e+05,227276.255641,3.056785e+05,258215.965344,89557.563452,0.0,374121.0,61456.0,112860.0,911.0,280085.0,607740.0,2275.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,2022,45820,1345,22.48,%,230878.0,5.369700e+04,30858.951001,6.751237e+04,32694.785366,18218.266001,0.0,36382.0,6889.0,11784.0,660.0,297947.0,89334.0,1987.4
872,2022,46140,2331,28.97,%,1006411.0,2.443900e+05,142723.974003,2.745027e+05,108317.909984,81929.391002,0.0,164229.0,31057.0,63747.0,3933.0,395327.0,377679.0,3138.0
873,2022,47260,8550,22.73,%,1760740.0,3.844730e+05,235602.113995,4.751754e+05,257462.399611,148211.222997,0.0,351364.0,61026.0,119400.0,3528.0,1261482.0,699550.0,11657.6
874,2022,48620,1516,20.64,%,643768.0,1.618890e+05,82246.350996,1.816031e+05,81941.549291,51666.979998,0.0,113437.0,26369.0,39928.0,1477.0,252086.0,253448.0,1747.2


In [138]:
aggregated_df.isna().sum()

Year               0
cbsacode           0
Sample_Size        0
Data_value         0
Data_value_unit    0
v051_rawvalue      0
v052_rawvalue      0
v009_rawvalue      0
v011_rawvalue      0
v049_rawvalue      0
v060_rawvalue      0
v070_numerator     0
v069_numerator     0
v023_numerator     0
v082_numerator     0
v062_numerator     0
v063_rawvalue      0
v067_numerator     0
v045_rawvalue      0
dtype: int64

In [139]:
aggregated_df[aggregated_df.duplicated()]

,Year,cbsacode,Sample_Size,Data_value,Data_value_unit,v051_rawvalue,v052_rawvalue,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue


In [140]:
aggregated_df.cbsacode.value_counts()

cbsacode
10740    12
30780    12
38900    12
38860    12
38300    12
         ..
20260    12
19820    12
19780    12
19740    12
49340    12
Name: count, Length: 73, dtype: int64

## Merge GDP,noise and Park

In [17]:
import pandas as pd
aggregated_df = pd.read_csv('../00_data/02_intermediate/20250602agg_smart_chr.csv')

In [18]:
# Step 1: Find CBSAs that appear in both 2011 and 2012
cbsa_with_both_years = aggregated_df[aggregated_df['Year'].isin([2011, 2012])].groupby('cbsacode')['Year'].nunique()
cbsa_with_both_years = cbsa_with_both_years[cbsa_with_both_years == 2].index

# Step 2: Filter original DataFrame to keep only those CBSAs
aggregated_df = aggregated_df[aggregated_df['cbsacode'].isin(cbsa_with_both_years)]

In [19]:
aggregated_df

,Year,cbsacode,v051_rawvalue,v052_rawvalue,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,v023_numerator,v082_numerator,v062_numerator,v063_rawvalue,v067_numerator,v045_rawvalue,Data_value,Sample_Size
1,2011,10420.0,542405.0,123668.340,93797.011840,117246.264800,74535.125480,44804.822620,0.0,94515.0,29077.0,41007.0,375.0,49528.0,223355.0,364.6,16.600000,112.0
2,2012,10420.0,542405.0,123668.340,86259.751960,121433.631400,73697.652160,43967.349300,0.0,90784.0,28913.0,40388.0,375.0,45768.0,221759.0,368.3,20.180000,118.0
3,2011,10740.0,841428.0,207848.321,119645.415194,132813.285121,83828.102769,37536.718343,0.0,142157.0,29683.0,72115.0,174.0,146437.0,298296.0,1024.7,21.295516,698.0
4,2012,10740.0,768515.0,188890.941,104113.349277,119447.357021,73239.331013,35823.223199,0.0,131916.0,31877.0,68841.0,171.0,100183.0,285827.0,878.4,21.340000,609.0
5,2011,10900.0,109638.0,25655.292,18056.282220,23095.244700,13353.250572,6802.599348,0.0,18230.0,5300.0,6087.0,20.0,74721.0,43437.0,90.1,11.850000,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,2012,48620.0,490864.0,133024.144,72999.330624,105562.757520,50813.259552,33636.946464,0.0,81464.0,21981.0,43564.0,56.0,45996.0,196327.0,558.5,16.370000,300.0
269,2011,49340.0,803701.0,189673.436,123419.540364,149208.698052,122191.485236,50350.260248,0.0,139705.0,37408.0,51125.0,447.0,66389.0,318623.0,206.2,17.340000,468.0
270,2012,49340.0,803701.0,189673.436,120963.430108,156577.028820,123419.540364,52806.370504,0.0,138687.0,38039.0,51058.0,447.0,61079.0,313894.0,242.5,19.090000,549.0
272,2011,49660.0,236735.0,51371.495,39853.153575,52087.144905,29287.433790,19833.895035,0.0,31633.0,14448.0,20784.0,63.0,40508.0,89013.0,376.9,16.120000,110.0


In [20]:
gdp = pd.read_csv('../00_data/02_intermediate/gdp_cbsa.csv')
park = pd.read_csv('../00_data/02_intermediate/cbsa_park.csv')
noise = pd.read_csv('../00_data/02_intermediate/noise_cbsa.csv')

In [21]:
#merge with gdp
gdp = gdp.rename(columns={"year": "Year"})
merged_result_1 = pd.merge(aggregated_df,gdp)

#merge with park
park = park.rename(columns={"CBSA Code": "cbsacode"})
merged_result_2 = pd.merge(
    merged_result_1,
    park[['cbsacode', 'Park_Area_Acres', 'Weighted_RESP']],
    on='cbsacode',
    how='left'
)

#merge with noise
noise['noise70n'] = noise['noise7080n'] + noise['noise8090n'] + noise['noise90n']
noise['noise80n'] = noise['noise8090n'] + noise['noise90n']
merged_result = pd.merge(
    merged_result_2,
    noise[['cbsacode', 'noise70n', 'noise80n', 'noise90n']],
    on='cbsacode',
    how='left'
)

In [22]:
merged_result

,Year,cbsacode,v051_rawvalue,v052_rawvalue,v009_rawvalue,v011_rawvalue,v049_rawvalue,v060_rawvalue,v070_numerator,v069_numerator,...,v067_numerator,v045_rawvalue,Data_value,Sample_Size,gdp,Park_Area_Acres,Weighted_RESP,noise70n,noise80n,noise90n
0,2011,10420.0,542405.0,123668.340,93797.011840,117246.264800,74535.125480,44804.822620,0.0,94515.0,...,223355.0,364.6,16.600000,112.0,30090713.0,41598.151360,0.216675,5494.820512,537.544918,53.114390
1,2012,10420.0,542405.0,123668.340,86259.751960,121433.631400,73697.652160,43967.349300,0.0,90784.0,...,221759.0,368.3,20.180000,118.0,31046119.0,41598.151360,0.216675,5494.820512,537.544918,53.114390
2,2011,10740.0,841428.0,207848.321,119645.415194,132813.285121,83828.102769,37536.718343,0.0,142157.0,...,298296.0,1024.7,21.295516,698.0,37141102.0,36984.815680,0.145476,5540.745134,1224.879163,17.445337
3,2012,10740.0,768515.0,188890.941,104113.349277,119447.357021,73239.331013,35823.223199,0.0,131916.0,...,285827.0,878.4,21.340000,609.0,37226565.0,36984.815680,0.145476,5540.745134,1224.879163,17.445337
4,2011,10900.0,109638.0,25655.292,18056.282220,23095.244700,13353.250572,6802.599348,0.0,18230.0,...,43437.0,90.1,11.850000,75.0,36883410.0,104714.633800,0.239067,6179.837741,694.010695,86.917561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,2012,48620.0,490864.0,133024.144,72999.330624,105562.757520,50813.259552,33636.946464,0.0,81464.0,...,196327.0,558.5,16.370000,300.0,28623784.0,10427.746640,0.313956,5302.334212,361.131273,60.846898
230,2011,49340.0,803701.0,189673.436,123419.540364,149208.698052,122191.485236,50350.260248,0.0,139705.0,...,318623.0,206.2,17.340000,468.0,36014006.0,52331.852940,0.206128,6650.566990,1171.716480,28.399980
231,2012,49340.0,803701.0,189673.436,120963.430108,156577.028820,123419.540364,52806.370504,0.0,138687.0,...,313894.0,242.5,19.090000,549.0,37033118.0,52331.852940,0.206128,6650.566990,1171.716480,28.399980
232,2011,49660.0,236735.0,51371.495,39853.153575,52087.144905,29287.433790,19833.895035,0.0,31633.0,...,89013.0,376.9,16.120000,110.0,15781356.0,5621.076757,0.203079,1981.787831,130.858132,15.124541


In [23]:
merged_result.sort_values(by=['cbsacode'], ascending=True).to_csv(f"{intermediate_dir}20250602agg_all.csv", index=False)